<a href="https://colab.research.google.com/github/bimapamungkas666/Sistem-Rekomendasi/blob/main/sistem_rekomendasi_0016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

df = pd.read_csv("https://raw.githubusercontent.com/bimapamungkas666/Sistem-Rekomendasi/main/data.csv")
df.head()

,Mitra,Alamat,Modul
0,Bima Pamungkas,"JL. Jetis Trawas RT 01 RW 003, Cepoko, Gunungp...",Implementasi Basis Data Web Service Sistem Bas...
1,PT Telkom Indonesia (Persero) Tbk,"Jl Japati 1, Bandung",Content Review Job Performance Strategic Commu...
2,PT STECHOQ ROBOTIKA INDONESIA,"Jalan Belimbing A-17 Perum Sidoarum Blok II, G...",Lighting Rendering Sclupting 3D Shadering Text...
3,PT Bank Negara Indonesia (Persero) Tbk,"Gedung Grha BNI, Jl. Jend. Sudirman No.10, RT....",Can do attitude Blender Unity Microservices Wi...
4,PT Zona Edukasi Nusantara,"Graha Aktiva, Ground Floor Jalan HR Rasuna Sai...",Kemampuan Literasi Kemampuan Numerasi Komunika...


## 1. Ikhtisar

In [13]:
df.describe()

,Mitra,Alamat,Modul
count,15,15,15
unique,15,15,15
top,Bima Pamungkas,"JL. Jetis Trawas RT 01 RW 003, Cepoko, Gunungp...",Implementasi Basis Data Web Service Sistem Bas...
freq,1,1,1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Mitra   15 non-null     object
 1   Alamat  15 non-null     object
 2   Modul   15 non-null     object
dtypes: object(3)
memory usage: 488.0+ bytes


## 2. Tampilan Modul Pembelajaran (Sebelum Preprocessing)

In [15]:
def print_description(index):
    example = df[df.index == index][['Modul', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [16]:
print_description(1)

Content Review Job Performance Strategic Communication Plan Content Design Content Delivery Content Strategy
Mitra: PT Telkom Indonesia (Persero) Tbk
Alamat: Jl Japati 1, Bandung


In [17]:
print_description(11)

Sofware engineering menerapkan proses pengembangan software menggunakan code editor vscode terminal version control dan basic networking
Mitra: PT Ruang Raya Indonesia
Alamat: Jl. Dr. Saharjo No.161, Manggarai Selatan, Tebet, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12860


In [18]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining 
Mitra: Bima Pamungkas
Alamat: JL. Jetis Trawas RT 01 RW 003, Cepoko, Gunungpati, Semarang


## 3. Text Preprocessing

In [19]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modul_pembelajaran_clean'] = df['Modul'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df.head()

,Mitra,Alamat,Modul,modul_pembelajaran_clean
0,Bima Pamungkas,"JL. Jetis Trawas RT 01 RW 003, Cepoko, Gunungp...",Implementasi Basis Data Web Service Sistem Bas...,implementasi basis data web service sistem bas...
1,PT Telkom Indonesia (Persero) Tbk,"Jl Japati 1, Bandung",Content Review Job Performance Strategic Commu...,content review job performance strategic commu...
2,PT STECHOQ ROBOTIKA INDONESIA,"Jalan Belimbing A-17 Perum Sidoarum Blok II, G...",Lighting Rendering Sclupting 3D Shadering Text...,lighting rendering sclupting 3d shadering text...
3,PT Bank Negara Indonesia (Persero) Tbk,"Gedung Grha BNI, Jl. Jend. Sudirman No.10, RT....",Can do attitude Blender Unity Microservices Wi...,can do attitude blender unity microservices wi...
4,PT Zona Edukasi Nusantara,"Graha Aktiva, Ground Floor Jalan HR Rasuna Sai...",Kemampuan Literasi Kemampuan Numerasi Komunika...,kemampuan literasi kemampuan numerasi komunika...


## 4. Tampilan Modul Pembelajaran (Setelah Preprocessing)

In [21]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modul_pembelajaran_clean', 'Mitra','Alamat']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Mitra:', example[1])
        print('Alamat:', example[2])

In [22]:
print_description_clean(1)

content review job performance strategic communication plan content design content delivery content strategy
Mitra: PT Telkom Indonesia (Persero) Tbk
Alamat: Jl Japati 1, Bandung


In [23]:
print_description_clean(11)

sofware engineering menerapkan proses pengembangan software menggunakan code editor vscode terminal version control dan basic networking
Mitra: PT Ruang Raya Indonesia
Alamat: Jl. Dr. Saharjo No.161, Manggarai Selatan, Tebet, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12860


In [24]:
print_description(0)

Implementasi Basis Data Web Service Sistem Basis Data Pemrograman Web Jaringan Komputer Sistem Operasi Pemrograman Aplikasi Mobile Text Mining 
Mitra: Bima Pamungkas
Alamat: JL. Jetis Trawas RT 01 RW 003, Cepoko, Gunungpati, Semarang


## 5. TF-IDF & Cosine Similarity

In [25]:
df.set_index('Mitra', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modul_pembelajaran_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.04723323, 0.        , 0.03726567,
        0.        , 0.        , 0.04537467, 0.0210648 , 0.04981889],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.01940793, 0.02154918, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.02407928,
        0.        , 0.        , 0.        , 0.01010533, 0.01020408,
        0.        , 0.01160064, 0.        , 0.        , 0.01242543],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.02407928, 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.01715328, 0.        ,
        0.        , 0.        , 0.06537348, 

In [26]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:50]

0                                        Bima Pamungkas
1                     PT Telkom Indonesia (Persero) Tbk
2                         PT STECHOQ ROBOTIKA INDONESIA
3                PT Bank Negara Indonesia (Persero) Tbk
4                             PT Zona Edukasi Nusantara
5                               PT Global Tiket Network
6                 PT. IlmuKomputerCom Braindevs Sistema
7                  PT RPN Pusat Penelitian Kelapa Sawit
8                     PT Hacktivate Teknologi Indonesia
9                           PT LENTERA BANGSA BENDERANG
10    PT Panji Edukasi Nusantara / Apple Developer A...
11                              PT Ruang Raya Indonesia
12                         PT Nurul Fikri Cipta Inovasi
13                              PT Marka Kreasi Persada
14                         PT Greatedu Global Mahardika
Name: Mitra, dtype: object

## 6. Modelling

In [27]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_modul = []
    
    # Mengambil nama modul berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 3 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_modul.append(list(df.index)[i])
        
    return recommended_modul

## 7. Prediksi

In [29]:
recommendations('Bima Pamungkas')

['PT Greatedu Global Mahardika',
 'PT RPN Pusat Penelitian Kelapa Sawit',
 'PT Nurul Fikri Cipta Inovasi']

In [30]:
recommendations("PT Telkom Indonesia (Persero) Tbk")

['PT. IlmuKomputerCom Braindevs Sistema',
 'PT Global Tiket Network',
 'Bima Pamungkas']

In [31]:
recommendations("PT Panji Edukasi Nusantara / Apple Developer Academy")

['PT Hacktivate Teknologi Indonesia',
 'Bima Pamungkas',
 'PT Telkom Indonesia (Persero) Tbk']